![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like Thanksgiving, and Christmas. 

In this project, I have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. I will be working with two data sources: grocery sales and complementary data. I have been provided with the `grocery_sales` table in csv format with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, I have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

I was requested to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, I will have to analyze monthly sales of Walmart and store the results of my analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, saving the `clean_data` and `agg_data` as the csv files.

In [4]:
import pandas as pd
import os
def extract(store_data, extra_data):
    original_df = pd.read_csv(store_data)
    extra_df = pd.read_parquet(extra_data)
    merged_df = original_df.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract("grocery_sales.csv", "extra_data.parquet")

In [5]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
  # Write your code here
    num_cols = raw_data.select_dtypes(include=['number']).columns
    raw_data[num_cols] = raw_data[num_cols].fillna(raw_data[num_cols].mean())
    
    raw_data['Month'] = pd.to_datetime(raw_data['Date']).dt.month

    # Filter rows where 'Weekly_Sales' is over 10,000
    raw_data = raw_data[raw_data['Weekly_Sales'] > 10000]

    # Drop unnecessary columns (adjust this list as needed)
    cols_to_drop = ['Date', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5',
                   'Fuel_Price', 'Temperature', 'Type', 'Size', 'index'] 
    raw_data = raw_data.drop(columns=cols_to_drop)

    return raw_data

In [6]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)

In [7]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    # Write your code here
    newdf = round(clean_data.groupby(['Month']).agg(Avg_Sales=('Weekly_Sales','mean')).reset_index(), 2)
    return newdf

In [8]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
print(agg_data)

    Month  Avg_Sales
0     1.0   33174.18
1     2.0   34333.33
2     3.0   33240.16
3     4.0   33420.72
4     5.0   33335.33
5     6.0   34582.47
6     7.0   33922.76
7     8.0   33644.79
8     9.0   33258.06
9    10.0   32736.99
10   11.0   36594.03
11   12.0   39238.80


In [9]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    # Write your code here
    full_data.to_csv(full_data_file_path, index = False)
    agg_data.to_csv(agg_data_file_path, index = False)

In [10]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths    
load(clean_data, 'clean_data.csv', agg_data, 'agg_data.csv')
#print(clean_data)

In [11]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    # Write your code here
    if os.path.isfile(file_path):
        return 'File exists'
    else:
        return 'File not found'

In [12]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation('clean_data.csv')
pass
validation('agg_data.csv')

'File exists'